### Получаем новости для учета новостного шума. **В лишний раз не заупскать ячейку!**
### Получаем описание статьи + дату и время публикации. Дату и время публикации переводим в дату, чтобы можно было объединить с датафреймом, где содержится информация по акциям.
### Сервис имеет ограничения, поэтому берем всего n последних дней
### Новости собираем по каждой отдельной акции по ключевому слову

In [ ]:
from datetime import datetime, timedelta
import requests
import pandas as pd
import time

API_KEY = 'PRIVATE_API_TOKEN'
STOCKS = ["goog"]
n_days = 2000
dt = datetime.strptime('2025-10-15', '%Y-%m-%d')

def generate_dates(n, dt):
    today = dt
    return [(today - timedelta(days=i)).strftime('%Y-%m-%d') for i in range(n)]

dates_list = generate_dates(n_days, dt)
lst_data_news_stock = []
dct = {}
for stock in STOCKS:
    lst_news_date = []
    for date in dates_list:
        url = f'https://gnews.io/api/v4/search?q=Google&lang=en&expand=content&from={date}T00:00:00.000Z&to={date}T23:59:59.999Z&max=5&apikey={API_KEY}'

        response = requests.get(url)
        data_news = response.json()
        articles = data_news.get("articles", [])
        if len(articles):
            for article in articles:
                data_news = pd.DataFrame([{'content': article['content'], 'publishedAt': article['publishedAt'], 'title': article['title'], 'description': article['description']}])
                data_news['publishedAt'] = pd.to_datetime(data_news['publishedAt']).dt.date
                data_news.columns = [f"{stock}_{col}".lower() for col in data_news.columns]
                data_news.to_csv('./csv/my_csv_2.csv', mode='a', header=True)
        print(f"{date} | {response}")
        time.sleep(2)

### Предобрабатываем dataframe с новостями

In [ ]:
import pandas as pd

df = pd.read_csv('./csv/my_csv_2.csv')
df = df[df['goog_publishedat'] != 'goog_publishedat'].drop_duplicates()
df[['goog_publishedat', 'goog_description']].to_csv('./csv/news_google_description.csv', mode='w', header=True)